In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import joblib

# setup
process = 'NonCodify'
inputs = pd.read_excel(f'../files/NonCodify/12 - December 2022 Monthly Summary.xlsx', engine='openpyxl')
inputs.fillna('0', inplace = True)

le_payer = LabelEncoder()
le_coding_tool = LabelEncoder()
le_shs_status = LabelEncoder()
le_pic_desc = LabelEncoder()
le_status = LabelEncoder()
le_category = LabelEncoder()
le_comment = LabelEncoder()

le_payer.classes_ = np.load('payer_classes.npy', allow_pickle=True)
le_coding_tool.classes_ = np.load('coding_tool_classes.npy', allow_pickle=True)
le_shs_status.classes_ = np.load('shs_status_classes.npy', allow_pickle=True)
le_pic_desc.classes_ = np.load('pic_desc_classes.npy', allow_pickle=True)
le_status.classes_ = np.load('status_classes.npy', allow_pickle=True)
le_category.classes_ = np.load('category_classes.npy', allow_pickle=True)
le_comment.classes_ = np.load('comment_classes.npy', allow_pickle=True)

inputs = inputs.fillna('0')
inputs['ACK'] = inputs['ACK'].astype('int64')
inputs['Form Letter'] = inputs['Form Letter'].astype('int64')
inputs['Proofs'] = inputs['Proofs'].astype('int64')
df = inputs[['ACK', 'Form Letter', 'Proofs']]
df['payer_n'] = le_payer.transform(inputs['PAYER'])
df['coding_tool_n'] = le_coding_tool.transform(inputs['Coding Tool'])
df['shs_status_n'] = le_shs_status.transform(inputs['RetrievalStatus'])
df['pic_desc_n'] = le_pic_desc.transform(inputs['Name'])

status = joblib.load('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/ML Training/models/noncodify_audit_status.joblib')
category = joblib.load('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/ML '
                       'Training/models/noncodify_audit_category.joblib')
comments = joblib.load('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/ML Training/models/noncodify_audit_comment.joblib')

In [ ]:
df['status_n'] = status.predict(df)
inputs['Status'] = le_status.inverse_transform(df['status_n'])
df['category_n'] = category.predict(df)
inputs['Category'] = le_category.inverse_transform(df['category_n'])
df['comment_n'] = comments.predict(df)
inputs['Comments'] = le_comment.inverse_transform(df['comment_n'])

In [ ]:
inputs.to_excel('12 - December 2022 Monthly Summary.xlsx')